In [1]:
import sys
sys.path.append('../modules/')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pulp
import seaborn as sns
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from mymodule_3 import *
from mymodule3_2 import *
from pulp_func_3 import *
df2023_result = pd.read_excel('../excels/附件3/2023_result.xlsx')
# 去除列名末尾的空格。很奇怪
df2023_result.columns = [column.rstrip(' ') for column in df2023_result.columns]
# print(df2023_result)

In [2]:
# 数据准备

merged_df = get_merged_df(df21, df22)
# print(merged_df)
total_yield_2023 = merged_df.groupby(['作物编号', '作物名称'])['单地块类型总产量/斤'].sum().reset_index()
total_yield_2023.columns = ['作物编号', '作物名称', '总产量/斤']
# print(total_yield_2023)
expected_sales_data_2023 = total_yield_2023.copy()
expected_sales_data_2023.columns = ['作物编号', '作物名称', '预期销售量/斤']

# 按照 '作物编号' 和 '作物名称' 进行分组，并对 '利润' 进行求和
total_profit = merged_df.groupby(['作物编号', '作物名称'])[['利润', '每亩利润']].sum().reset_index()
# 重命名列，以便清晰地表示数据
total_profit.columns = ['作物编号', '作物名称', '总利润/元', '每亩总利润/元']
# 打印 total_profit
# print(total_profit)
total_profit_2023 = total_profit['总利润/元'].sum()
# print("总利润/元的和:", total_profit_2023)
std_devs = [0.005, 0.005, 0.01, 0.01, 0.003, 0.003, 0.003, 0.003]
past_profits = [total_profit_2023]

fields = create_fields(df11)
# for field in fields:
#     print(field)
new_fields = create_new_fields(fields)
# for field in new_fields:
#     print(field)

expected_sales_data = total_yield_2023.copy() # 初始值为2023年的数据
expected_sales_data.columns = ['作物编号', '作物名称', '预期销售量/斤']
crops = create_crops(df12, df22)

In [3]:
mc = 1 # 蒙特卡洛模拟次数
weights = [0.7, 0.3] # 分数计算权重
k = 0.5
min_area_percent = 0.2
max_plots = 4
total_score_max = 0
total_score_i_list = []
scores_max = []
profits_max = []
total_profit_i_list = []
stability_max = []
total_stability_i_list = []
for i in range(mc): 
    # 每次蒙特卡洛模拟都要重新初始化数据
    df_template_list = []
    scores = [total_profit_2023, 0, 0, 0, 0, 0, 0, 0] # 2023年到2030年的得分
    profits = [total_profit_2023, 0, 0, 0, 0, 0, 0, 0] # 2023年到2030年的利润
    past_profits = [total_profit_2023] # 别忘了重置这个！其实这个和上面的profits是一样的，但已经这么设定了就不改了
    stabilitys = [0, 0, 0, 0, 0, 0, 0, 0] # 2023年到2030年的稳定性分数（其实是不稳定性）
    expected_sales_data = total_yield_2023.copy() # 初始值为2023年的数据
    expected_sales_data.columns = ['作物编号', '作物名称', '预期销售量/斤']
    crops = create_crops(df12, df22)
    # 2024年

    # 更新数据
    expected_sales_data = update_expected_sales_data(expected_sales_data, expected_sales_data_2023, std_devs)
    crops = update_crops(crops, std_devs)

    variables, binary_variables, actual_sales, excess_yield = create_variables(new_fields, crops)
    # 创建线性规划问题
    prob = pulp.LpProblem("Crop_Planting_Optimization", pulp.LpMaximize)
    # 定义目标函数的各个部分
    profit = define_objective_function(variables, actual_sales, excess_yield, crops, new_fields, k)
    # 将目标函数添加到问题中
    prob += profit
    prob = add_constraints2(prob, variables, binary_variables, actual_sales, excess_yield,
                            crops, new_fields, total_yield_2023, df2023_result, min_area_percent, max_plots)

    # 求解问题
    prob.solve()
    print("Status:", pulp.LpStatus[prob.status])
    update_new_fields(variables, new_fields)
    df_template = pd.read_excel('../excels/附件3/template.xlsx')
    df_template_list.append(update_template(df_template, new_fields))


    objective_value = pulp.value(prob.objective)
    past_profits.append(objective_value)
    print("Objective Value / Profit:", objective_value)
    profits[1] = objective_value

    stability = calculate_stability(past_profits)
    print("Stability:", stability)
    stabilitys[1] = stability

    score_now = calculate_score(objective_value, stability, weights)
    print("Score:", score_now)
    scores[1] = score_now
    # 清空当年的种植信息
    for field in new_fields:
        field.planted_crop = []

    # 2025年到2030年

    for year in range(2025, 2031):
        list = [k, min_area_percent, max_plots, year]
        expected_sales_data = update_expected_sales_data(expected_sales_data, expected_sales_data_2023, std_devs)
        crops = update_crops(crops, std_devs)

        df_last1_result = pd.read_excel(f'../excels/附件3/{year - 1}_result_{k}.xlsx')
        df_last1_result.columns = [column.rstrip(' ') for column in df_last1_result.columns]
        if year - 2 == 2023:
            df_last2_result = pd.read_excel(f'../excels/附件3/2023_result.xlsx')
        else:    
            df_last2_result = pd.read_excel(f'../excels/附件3/{year - 2}_result_{k}.xlsx')
        df_last2_result.columns = [column.rstrip(' ') for column in df_last2_result.columns]

        objective_value = optimize_planting_strategy(new_fields, crops, expected_sales_data, df_last1_result, df_last2_result, list)
        df_template_list.append(update_template(df_template, new_fields))
        past_profits.append(objective_value)
        print("Objective Value / Profit:", objective_value)
        profits[year - 2023] = objective_value

        stability = calculate_stability(past_profits)
        print("Stability:", stability)
        stabilitys[year - 2023] = stability

        score_now = calculate_score(objective_value, stability, weights)
        scores[year - 2023] = score_now
        print("Score:", score_now)
        # 清空当年的种植信息
        for field in new_fields:
            field.planted_crop = []
    
    print("\nScores:", scores)
    total_score_i = sum(scores)
    total_score_i_list.append(total_score_i)
    print("Total Score:", total_score_i)
    print("\n")
    total_profit_i = sum(profits)
    total_stability_i = sum(stabilitys)
    total_profit_i_list.append(total_profit_i)
    total_stability_i_list.append(total_stability_i)

    if total_score_i > total_score_max:
        total_score_max = total_score_i
        scores_max = scores
        profits_max = profits
        stability_max = stabilitys
        for template in df_template_list:
            output(template, year, k)

print("Total Score Max:", total_score_max)
print("Total Score i List:", total_score_i_list)
print("Scores Max:", scores_max)
print("Profits Max:", profits_max)
print("Total Profit i List:", total_profit_i_list)
print("Stability Max:", stability_max)
print("Total Stability i List:", total_stability_i_list)

Status: Optimal


e:\Users\Timothy\anaconda3\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


Objective Value / Profit: 8707189.746542927
Stability: 0.0
Score: 6095032.822580049
Status: Infeasible
Objective Function Value: -7140461.337796753
Objective Value / Profit: -7140461.337796753
Stability: 9314246.290441304
Score: -7792596.823590118
Status: Infeasible
Objective Function Value: 8864073.145138923
Objective Value / Profit: 8864073.145138923
Stability: 13065852.514799856
Score: 2285095.447157289


KeyboardInterrupt: 